In [68]:
import pandas as pd
#xlsm = 'razao.xlsm'
razao = pd.read_excel(xlsm, sheet_name=0)
posicao = pd.read_excel(xlsm, sheet_name=1)
pdd = pd.read_excel(xlsm, sheet_name=2)

In [69]:
razao_dict = dict(zip(razao["Código Empresa"], razao["Descrição Empresa"]))
posicao_dict = dict(zip(posicao["Empresa"], posicao["Nome Completo"]))
pdd_dict = dict(zip(pdd["Empresa"], pdd["Nome Completo"]))

In [70]:
codigo_empresa = {}
for d in [razao_dict, posicao_dict, pdd_dict]:
    for key, value in d.items():
        if key not in codigo_empresa:
            codigo_empresa[key] = value

In [71]:
dc = razao.groupby('Código Empresa')[['Débito', 'Crédito']].sum().reset_index()

In [72]:
posicao_grouped = posicao.groupby('Empresa')['Saldo'].sum().reset_index()


In [73]:
merged_df = pd.merge(posicao_grouped, dc, left_on='Empresa', right_on='Código Empresa', how='outer')


In [74]:
merged_df.loc[merged_df['Empresa'].isna(), 'Empresa'] = merged_df['Código Empresa']

In [75]:
pdd_grouped = pdd.groupby('Empresa')['Saldo'].sum().reset_index()

In [76]:
pdd_grouped.rename(columns={'Empresa': 'Código', 'Saldo': 'PDD'}, inplace=True)

In [80]:
pdd_grouped

,Código,PDD
0,56390.0,16215.93
1,57123.0,18207.18


In [77]:
merged_pdd = pd.merge(merged_df, pdd_grouped, left_on='Empresa', right_on='Código', how='outer')


In [78]:
merged_pdd.loc[merged_df['Empresa'].isna(), 'Empresa'] = merged_pdd['Código']

In [79]:
merged_pdd.drop(columns=['Código Empresa', 'Código'], inplace=True)


In [80]:
merged_pdd[['Saldo', 'Débito', 'Crédito', 'PDD']] = merged_pdd[['Saldo', 'Débito', 'Crédito', 'PDD']].fillna(0.0)


In [81]:
merged_pdd['Diferença'] = merged_pdd['Saldo'] - (merged_pdd['Débito'] - merged_pdd['Crédito'] - merged_pdd['PDD'])


In [82]:
columns_to_round = ['Saldo', 'Débito', 'Crédito', 'PDD', 'Diferença']
merged_pdd[columns_to_round] = merged_pdd[columns_to_round].round(2)

In [83]:
def check_sum_difference_between_dfs(df1, col1, df2, col2):
    sum_col1 = df1[col1].sum()
    sum_col2 = df2[col2].sum()
    difference = abs(sum_col1 - sum_col2)
    return difference < 1, difference


In [84]:
comparisons = [
    ('Débito', 'Débito', razao),
    ('Crédito', 'Crédito', razao),
    ('Saldo', 'Saldo', posicao),
    ('PDD', 'Saldo', pdd)
]

all_match = True
for col1, col2, df2 in comparisons:
    is_match, diff = check_sum_difference_between_dfs(merged_pdd, col1, df2, col2)
    if not is_match:
        all_match = False
        break

if all_match:
    # Write merged_pdd to an Excel file
    merged_pdd.to_excel('merged_pdd.xlsx', index=False)
    print("Data written to merged_pdd.xlsx")
else:
    raise ValueError("Not matching")

Data written to merged_pdd.xlsx


In [60]:
merged_pdd["Nome Cliente"] = merged_pdd["Empresa"].map(codigo_empresa)


In [62]:
merged_pdd = merged_pdd[["Empresa", "Nome Cliente", "Saldo", "Débito", "Crédito", "PDD", "Diferença"]]


In [66]:
def round_small_values(value):
    return 0.0 if value < 0.01 else value

merged_pdd["Diferença"] = merged_pdd["Diferença"].apply(round_small_values)


In [67]:
merged_pdd

,Empresa,Nome Cliente,Saldo,Débito,Crédito,PDD,Diferença
0,28.0,A FONSECA E CIA LTDA,10201.08,133304.86,123103.78,0.0,0.0
1,92.0,ANTONIO LOBASCZ E CIA LTDA,0.00,184678.36,184678.36,0.0,0.0
2,112.0,BRASIL ESCOLAR LTDA,16840.09,112126.80,95286.71,0.0,0.0
3,152.0,J D CAMARGO E CIA LTDA,0.00,4865.04,4865.04,0.0,0.0
4,154.0,ARNALDO COMERCIO E REPRESENTAC,0.00,9048.34,9048.34,0.0,0.0
...,...,...,...,...,...,...,...
8085,800186.0,TW TRANSPORTES E LOGISTICA LTDA,0.00,944.30,944.30,0.0,0.0
8086,802494.0,RODOVIARIO BEDIN LTDA,0.00,14059.50,14059.50,0.0,0.0
8087,803776.0,RTC LOGISTICA E TRANSPORTES LTDA,2653.81,2653.81,0.00,0.0,0.0
8088,804115.0,TRANSPORTES JESUS LTDA,484.44,2024.94,1540.50,0.0,0.0
